In [32]:
import pandas as pd
import fbprophet as ph

Importing plotly failed. Interactive plots will not work.


To complete our analysis and conclude about the ecoonomic impact that COVID-19 has on different regions, we will compute the forecasting of our response variable (difference in EDI) 1 year into the future (2021-07-01). Note that this difference is calculated by subtracting actual EDI (with the presence of COVID) from projected EDI (assuming no pandemic). Therefore, a positive difference highlights the negative ecomonic impact. 

Vice versa, to calculate the actual EDI 1 year from mow, we need to subtract the difference from the projected EDI. Here we can project EDI using the historical data using the same methodolgy we applied before. To compute the difference, we will have to project what each predictor is 1 year from now, and feed the values into the multi-linear regression model we developed in R. 

For demonstration purpose, we will use just USA

In [2]:
# import the complete dataset
predictor = pd.read_csv('./final_data-2.csv')

In [4]:
predictor.columns

Index(['country_region', 'date',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'Country', 'Population',
       'Pop. Density (per sq. mi.)', 'Birthrate', 'Deathrate', 'Agriculture',
       'Industry', 'Service', 'total_cases', 'total_cases_per_million',
       'new_cases_per_million', 'new_deaths_per_million', 'stringency_index',
       'population', 'population_density', 'gdp_per_capita',
       'hospital_beds_per_thousand'],
      dtype='object')

In [15]:
# subset US data
US_data = predictor.loc[predictor['country_region'] == 'United States', ]

In [16]:
US_data

,country_region,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,Country,Population,...,Service,total_cases,total_cases_per_million,new_cases_per_million,new_deaths_per_million,stringency_index,population,population_density,gdp_per_capita,hospital_beds_per_thousand
1162,United States,2/15/2020,6,2,15,3,2,-1,United States,298444215,...,0.787,15,0.045,0.000,0.000,5.56,331002647,35.608,54225.446,2.77
1163,United States,2/16/2020,7,1,16,2,0,-1,United States,298444215,...,0.787,15,0.045,0.000,0.000,5.56,331002647,35.608,54225.446,2.77
1164,United States,2/17/2020,6,0,28,-9,-24,5,United States,298444215,...,0.787,15,0.045,0.000,0.000,5.56,331002647,35.608,54225.446,2.77
1165,United States,2/18/2020,0,-1,6,1,0,1,United States,298444215,...,0.787,15,0.045,0.000,0.000,5.56,331002647,35.608,54225.446,2.77
1166,United States,2/19/2020,2,0,8,1,1,0,United States,298444215,...,0.787,15,0.045,0.000,0.000,5.56,331002647,35.608,54225.446,2.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,United States,7/25/2020,-19,-2,77,-21,-13,4,United States,298444215,...,0.787,4112529,12424.460,236.938,3.940,68.98,331002647,35.608,54225.446,2.77
1324,United States,7/26/2020,-17,-5,65,-24,-16,3,United States,298444215,...,0.787,4178027,12622.337,197.878,2.761,68.98,331002647,35.608,54225.446,2.77
1325,United States,7/27/2020,-14,-4,48,-32,-38,10,United States,298444215,...,0.787,4234020,12791.499,169.162,1.435,68.98,331002647,35.608,54225.446,2.77
1326,United States,7/28/2020,-14,-3,53,-32,-38,11,United States,298444215,...,0.787,4290263,12961.416,169.917,3.251,68.98,331002647,35.608,54225.446,2.77


In [18]:
# subset predictors so they match the data needed in R
US_data = US_data.loc[:, ['date', 'hospital_beds_per_thousand','Birthrate','population_density','Agriculture','Service',
                      'Deathrate', 'Industry','retail_and_recreation_percent_change_from_baseline', 'new_cases_per_million',
                      'stringency_index','new_deaths_per_million']]

In [19]:
US_data

,date,hospital_beds_per_thousand,Birthrate,population_density,Agriculture,Service,Deathrate,Industry,retail_and_recreation_percent_change_from_baseline,new_cases_per_million,stringency_index,new_deaths_per_million
1162,2/15/2020,2.77,14.14,35.608,0.01,0.787,8.26,0.204,6,0.000,5.56,0.000
1163,2/16/2020,2.77,14.14,35.608,0.01,0.787,8.26,0.204,7,0.000,5.56,0.000
1164,2/17/2020,2.77,14.14,35.608,0.01,0.787,8.26,0.204,6,0.000,5.56,0.000
1165,2/18/2020,2.77,14.14,35.608,0.01,0.787,8.26,0.204,0,0.000,5.56,0.000
1166,2/19/2020,2.77,14.14,35.608,0.01,0.787,8.26,0.204,2,0.000,5.56,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
1323,7/25/2020,2.77,14.14,35.608,0.01,0.787,8.26,0.204,-19,236.938,68.98,3.940
1324,7/26/2020,2.77,14.14,35.608,0.01,0.787,8.26,0.204,-17,197.878,68.98,2.761
1325,7/27/2020,2.77,14.14,35.608,0.01,0.787,8.26,0.204,-14,169.162,68.98,1.435
1326,7/28/2020,2.77,14.14,35.608,0.01,0.787,8.26,0.204,-14,169.917,68.98,3.251


In [20]:
# define a prediction interval
interval_1 = len(pd.date_range(start = '7/30/2020', end = '7/1/2021'))

In [49]:
# save column names except for date
col_names = US_data.columns
col_names = [col for col in col_names if col != 'date']

In [86]:
len(col_names)

11

In [69]:
# create a dataframe for storing projected result
predictions_1 = pd.DataFrame(columns=col_names)

In [80]:
def forecast_predictors(df, columns, f_interval, predictions_df, date, freq = 'D'):
    '''
    input: df - a dataframe containing predictor's values
           columns - column names of df
           f_interval - prediction interval
           prediction_df - a dataframe that stores prediction result
           date - a specific date to extract in string
           freq - prediction frequency; default daily forecast
    '''
    for col in columns:
        col_df = df[['date', col]]
        col_df_names = col_df.columns
        col_df.columns = ['ds', 'y']
        m = ph.Prophet()
        m.fit(col_df)
        future = m.make_future_dataframe(periods = f_interval, freq = freq)
        forecast = m.predict(future)
        future_predict = forecast[['ds', 'yhat']]
        future_predict.columns = col_df_names
        future_prediction = future_predict.loc[future_predict['date'] == date, :]
        predictions_df[col] = future_prediction[col]

#     return future_predict

In [ ]:
# make prediction on 2021-07-01
forecast_predictors(US_data, col_names, interval_1, predictions_1, '2021-07-01')

In [82]:
predictions_1

,hospital_beds_per_thousand,Birthrate,population_density,Agriculture,Service,Deathrate,Industry,retail_and_recreation_percent_change_from_baseline,new_cases_per_million,stringency_index,new_deaths_per_million
502,2.77,14.14,35.608,0.01,0.787,8.26,0.204,84.638213,1219.388443,52.378784,-8.650716


In [85]:
# export as csv
predictions_1.to_csv('one_year_prediction.csv')

Now project EDI 1 year from now using historical data. The projeted EDI subtracts projected EDI difference, which will be return by R, gives the actual EDI (with presence of COVID)

In [ ]:
# define a prediction interval
new_prediction_interval = len(pd.date_range(start = '7/2/2020', end = '7/1/2021'))

In [ ]:
# read computed EDI data
edis = pd.read_csv('./EDI_indicators.csv')

In [ ]:
# subset US data
edis_US = edis.loc[edis['country'] == 'Untied States', :]
edis_US.drop(['country'], axis = 1, inplace = True),
edis_US

In [ ]:
def forecast_economic_indicator(df, f_interval, freq = 'D'):
    '''
    input: dataframe - a dataframe containing date and economic indicator value
           f_interval - prediction interval
           freq - prediction frequency; default daily forecast
    output: df - original dataframe with changed column names
            hist_predict - prediction on historical data in a dataframe
            future_predict - forecast on prediction interval in a dataframe
            df_names - column names of original dataframe
    '''
    # get dataframe column names
    df_names = df.columns
    # change dataframe column names
    df.columns = ['ds', 'y']
    # convert ds column name
    df['ds'] = pd.to_datetime(df['ds'])
    
    # initiate a prophet model
    m = ph.Prophet()
    # fit on dataframe
    m.fit(df)
    # define future prediction
    future = m.make_future_dataframe(periods = f_interval, freq = freq)
    # make prediction
    forecast = m.predict(future)
    # predict historical values
    hist_predict = forecast[['ds','yhat']][:-f_interval]
    # predict future values
    future_predict = forecast[['ds','yhat']][-f_interval:]
    
    return df, hist_predict, future_predict, df_names

In [ ]:
# make prediction
_,_,pre, _ = forecast_economic_indicator(edis_US, new_prediction_interval, freq = 'D')
pred